In [9]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from typing import Dict, List, Optional
import warnings
warnings.filterwarnings('ignore')


def load_and_prepare_yfinance_data_corrected(csv_file_path: str) -> pd.DataFrame:
    """
    Load and prepare the specific malformed yfinance CSV format
    """
    # Your CSV has 3 header rows:
    # Row 1: Price,Close,High,Low,Open,Volume  
    # Row 2: Ticker,USDINR=X,USDINR=X,USDINR=X,USDINR=X,USDINR=X
    # Row 3: Date,,,,,
    # Then actual data starts
    
    # Skip first 3 rows and assign proper column names
    df = pd.read_csv(
        csv_file_path, 
        skiprows=3, 
        names=['DATE', 'CLOSE PRICE', 'HIGH PRICE', 'LOW PRICE', 'OPEN PRICE', 'Volume']
    )
    
    # Convert DATE column to datetime
    df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')
    
    # Convert all price columns to numeric
    price_columns = ['OPEN PRICE', 'HIGH PRICE', 'LOW PRICE', 'CLOSE PRICE']
    for col in price_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Convert Volume to numeric
    df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')
    
    # Remove rows with invalid dates or prices
    df = df.dropna(subset=['DATE'] + price_columns)
    
    # Sort by date and reset index
    df = df.sort_values('DATE').reset_index(drop=True)
    
    print(f"Loaded {len(df)} rows of data from {df['DATE'].min()} to {df['DATE'].max()}")
    print(f"Available columns: {df.columns.tolist()}")
    print(f"Target column dtype: {df['OPEN PRICE'].dtype}")
    print(f"Sample data:\n{df[['DATE', 'OPEN PRICE', 'HIGH PRICE', 'LOW PRICE', 'CLOSE PRICE']].head()}")
    
    return df


def predict_last_n_days_with_ensemble_final(
    df: pd.DataFrame,
    n_days: int = 4,
    context_length: int = 10,
    target_column: str = 'OPEN PRICE',
    date_column: str = 'DATE',
    ensemble_models: list = None
) -> Dict:
    """
    Predict last n days with proper data type handling
    """
    
    if ensemble_models is None:
        ensemble_models = [
            {'model_size': 'tiny', 'weight': 0.15},
            {'model_size': 'mini', 'weight': 0.2},
            {'model_size': 'small', 'weight': 0.25},
            {'model_size': 'base', 'weight': 0.4}
        ]
    
    # Prepare data
    df = df.sort_values(date_column).reset_index(drop=True)
    
    # Ensure target column is proper numeric type
    target_series = df[target_column].astype(np.float32).values
    dates = pd.to_datetime(df[date_column])
    
    predictions_results = []
    actual_values = []
    prediction_dates = []
    
    try:
        from chronos import ChronosPipeline
        import torch
        
        # For each of the last n_days, predict using previous context_length days
        for day_offset in range(n_days):
            # Calculate indices for sliding window
            prediction_idx = len(df) - n_days + day_offset
            context_start = max(0, prediction_idx - context_length)
            context_end = prediction_idx
            
            # Extract context window
            context_data = target_series[context_start:context_end]
            actual_value = target_series[prediction_idx]
            prediction_date = dates.iloc[prediction_idx]
            
            # Ensure context data is float32
            context_data = context_data.astype(np.float32)
            
            # Store actual values for comparison
            actual_values.append(float(actual_value))
            prediction_dates.append(prediction_date)
            
            # Run ensemble predictions
            model_predictions = {}
            ensemble_preds = []
            weights = []
            
            print(f"Predicting day {day_offset + 1}/{n_days}: {prediction_date.strftime('%Y-%m-%d')}")
            print(f"  Context data shape: {context_data.shape}, dtype: {context_data.dtype}")
            
            for model_config in ensemble_models:
                try:
                    model_name = f"amazon/chronos-t5-{model_config['model_size']}"
                    pipeline = ChronosPipeline.from_pretrained(
                        model_name,
                        device_map="auto",
                        torch_dtype=torch.bfloat16,
                    )
                    
                    # Prepare context tensor with explicit float32 conversion
                    context_tensor = torch.tensor(context_data, dtype=torch.float32).unsqueeze(0)
                    
                    # Generate forecast
                    forecast = pipeline.predict(
                        context_tensor,
                        prediction_length=1,
                        num_samples=20
                    )
                    
                    # Extract prediction statistics
                    forecast_samples = forecast[0].cpu().numpy()
                    mean_pred = float(np.mean(forecast_samples))
                    p10 = float(np.percentile(forecast_samples, 10))
                    p50 = float(np.percentile(forecast_samples, 50))
                    p90 = float(np.percentile(forecast_samples, 90))
                    
                    model_predictions[f"chronos_{model_config['model_size']}"] = {
                        'prediction': mean_pred,
                        'confidence_intervals': {'p10': p10, 'p50': p50, 'p90': p90}
                    }
                    
                    ensemble_preds.append(mean_pred)
                    weights.append(model_config['weight'])
                    
                    print(f"  ✓ {model_config['model_size']}: {mean_pred:.4f}")
                    
                except Exception as e:
                    print(f"  ✗ {model_config['model_size']} failed: {str(e)}")
                    continue
            
            # Calculate ensemble prediction
            if ensemble_preds:
                ensemble_pred = float(np.average(ensemble_preds, weights=weights))
                model_predictions['ensemble'] = {
                    'prediction': ensemble_pred,
                    'individual_predictions': ensemble_preds,
                    'weights_used': weights
                }
                print(f"  🎯 Ensemble: {ensemble_pred:.4f} (Actual: {actual_value:.4f})")
            
            predictions_results.append({
                'date': prediction_date,
                'actual': float(actual_value),
                'predictions': model_predictions
            })
    
    except ImportError:
        print("Chronos not available, using statistical fallback...")
        predictions_results = create_statistical_predictions_n_days_final(
            df, n_days, context_length, target_column, date_column
        )
    
    return {
        'predictions': predictions_results,
        'dates': prediction_dates,
        'actuals': actual_values,
        'context_length': context_length,
        'n_days': n_days
    }


def create_statistical_predictions_n_days_final(
    df: pd.DataFrame,
    n_days: int,
    context_length: int,
    target_column: str,
    date_column: str
) -> List[Dict]:
    """Enhanced statistical predictions fallback"""
    
    target_series = df[target_column].astype(np.float32).values
    dates = pd.to_datetime(df[date_column])
    predictions_results = []
    
    print("Using statistical fallback methods...")
    
    for day_offset in range(n_days):
        prediction_idx = len(df) - n_days + day_offset
        context_start = max(0, prediction_idx - context_length)
        context_end = prediction_idx
        
        context_data = target_series[context_start:context_end]
        actual_value = float(target_series[prediction_idx])
        prediction_date = dates.iloc[prediction_idx]
        
        # Enhanced statistical methods
        ma_3 = float(np.mean(context_data[-3:]))
        ma_5 = float(np.mean(context_data[-5:]))
        ma_10 = float(np.mean(context_data[-10:]))
        ma_full = float(np.mean(context_data))
        
        # Weighted moving average (recent data has higher weight)
        weights = np.exp(np.linspace(-1, 0, len(context_data[-10:])))
        weights = weights / weights.sum()
        wma = float(np.average(context_data[-10:], weights=weights))
        
        # Exponential smoothing
        alpha = 0.3
        exp_smooth = float(context_data[-1])
        for i in range(len(context_data)-2, -1, -1):
            exp_smooth = alpha * float(context_data[i]) + (1 - alpha) * exp_smooth
        
        # Linear trend
        x = np.arange(len(context_data))
        coeffs = np.polyfit(x, context_data, 1)
        trend_pred = float(coeffs[0] * len(x) + coeffs[1])
        
        # Momentum-based prediction
        if len(context_data) >= 3:
            momentum = float(context_data[-1] + (context_data[-1] - context_data[-3]))
        else:
            momentum = float(context_data[-1])
        
        # Median-based robust prediction
        median_pred = float(np.median(context_data[-10:]))
        
        # Create ensemble from all statistical methods
        all_preds = [ma_3, ma_5, ma_10, wma, exp_smooth, trend_pred, momentum, median_pred]
        statistical_ensemble = float(np.mean(all_preds))
        
        # Robust ensemble (remove outliers)
        q75, q25 = np.percentile(all_preds, [75, 25])
        iqr = q75 - q25
        lower_bound = q25 - 1.5 * iqr
        upper_bound = q75 + 1.5 * iqr
        robust_preds = [p for p in all_preds if lower_bound <= p <= upper_bound]
        robust_ensemble = float(np.mean(robust_preds)) if robust_preds else statistical_ensemble
        
        model_predictions = {
            'moving_average_3': {'prediction': ma_3},
            'moving_average_5': {'prediction': ma_5},
            'moving_average_10': {'prediction': ma_10},
            'weighted_ma': {'prediction': wma},
            'exponential_smoothing': {'prediction': exp_smooth},
            'linear_trend': {'prediction': trend_pred},
            'momentum': {'prediction': momentum},
            'median_prediction': {'prediction': median_pred},
            'statistical_ensemble': {'prediction': statistical_ensemble},
            'robust_ensemble': {'prediction': robust_ensemble}
        }
        
        predictions_results.append({
            'date': prediction_date,
            'actual': actual_value,
            'predictions': model_predictions
        })
        
        print(f"  Day {day_offset + 1}/{n_days}: {prediction_date.strftime('%Y-%m-%d')}")
        print(f"    Statistical Ensemble: {statistical_ensemble:.4f}")
        print(f"    Robust Ensemble: {robust_ensemble:.4f}")
        print(f"    Actual: {actual_value:.4f}")
    
    return predictions_results


def create_prediction_plots_final(results: Dict, save_plots: bool = True) -> go.Figure:
    """Create enhanced interactive Plotly visualizations"""
    
    predictions_data = results['predictions']
    
    if not predictions_data:
        print("No prediction data available for plotting")
        return go.Figure()
    
    # Extract data for plotting
    dates = [p['date'] for p in predictions_data]
    actuals = [p['actual'] for p in predictions_data]
    
    # Get all model names
    all_models = set()
    for p in predictions_data:
        all_models.update(p['predictions'].keys())
    
    # Create subplot layout
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Predictions vs Actuals', 'Prediction Errors (%)', 'Model Performance (MAE)', 'Model Rankings'),
        specs=[[{'secondary_y': False}, {'secondary_y': False}],
               [{'secondary_y': False}, {'secondary_y': False}]]
    )
    
    # Plot 1: Predictions vs Actuals
    fig.add_trace(
        go.Scatter(x=dates, y=actuals, mode='lines+markers', name='Actual', 
                  line=dict(color='black', width=4), marker=dict(size=10)),
        row=1, col=1
    )
    
    # Color palette for models
    colors = px.colors.qualitative.Set3
    model_errors = {}
    model_mapes = {}
    
    for i, model in enumerate(sorted(all_models)):
        predictions = []
        errors = []
        percentage_errors = []
        
        for p in predictions_data:
            if model in p['predictions']:
                pred_value = p['predictions'][model]['prediction']
                predictions.append(pred_value)
                error = abs(pred_value - p['actual'])
                errors.append(error)
                percentage_errors.append((error / p['actual']) * 100)
            else:
                predictions.append(None)
                errors.append(None)
                percentage_errors.append(None)
        
        model_errors[model] = np.nanmean(errors) if errors else 0
        model_mapes[model] = np.nanmean(percentage_errors) if percentage_errors else 0
        
        # Plot predictions
        fig.add_trace(
            go.Scatter(x=dates, y=predictions, mode='lines+markers', 
                      name=f'{model}', line=dict(color=colors[i % len(colors)]),
                      marker=dict(size=6)),
            row=1, col=1
        )
        
        # Plot percentage errors
        fig.add_trace(
            go.Scatter(x=dates, y=percentage_errors, mode='lines+markers', 
                      name=f'{model} Error %', line=dict(color=colors[i % len(colors)]),
                      showlegend=False),
            row=1, col=2
        )
    
    # Plot 3: Model Comparison (MAE)
    model_names = list(model_errors.keys())
    mae_values = list(model_errors.values())
    
    if model_names:
        # Sort by performance
        sorted_models = sorted(zip(model_names, mae_values), key=lambda x: x[1])
        sorted_names, sorted_maes = zip(*sorted_models)
        
        fig.add_trace(
            go.Bar(x=list(sorted_names), y=list(sorted_maes), name='Mean Absolute Error',
                   marker_color=colors[:len(sorted_names)], showlegend=False),
            row=2, col=1
        )
    
    # Plot 4: Model Rankings (MAPE)
    if model_names:
        sorted_mapes = sorted(zip(model_names, [model_mapes[m] for m in model_names]), key=lambda x: x[1])
        sorted_mape_names, sorted_mape_values = zip(*sorted_mapes)
        
        fig.add_trace(
            go.Bar(x=list(sorted_mape_names), y=list(sorted_mape_values), name='MAPE %',
                   marker_color=px.colors.qualitative.Pastel[:len(sorted_mape_names)], showlegend=False),
            row=2, col=2
        )
    
    # Update layout
    fig.update_layout(
        title_text=f"USDINR Prediction Analysis - Last {results['n_days']} Days",
        title_x=0.5,
        height=900,
        showlegend=True,
        template='plotly_white',
        font=dict(size=10)
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_yaxes(title_text="USD/INR Rate", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=2)
    fig.update_yaxes(title_text="Error %", row=1, col=2)
    fig.update_xaxes(title_text="Model", row=2, col=1)
    fig.update_yaxes(title_text="MAE", row=2, col=1)
    fig.update_xaxes(title_text="Model", row=2, col=2)
    fig.update_yaxes(title_text="MAPE %", row=2, col=2)
    
    # Rotate x-axis labels for better readability
    fig.update_xaxes(tickangle=-45, row=2, col=1)
    fig.update_xaxes(tickangle=-45, row=2, col=2)
    
    if save_plots:
        fig.write_html("usdinr_prediction_analysis_final.html")
        print("Interactive plot saved as 'usdinr_prediction_analysis_final.html'")
    
    return fig


def calculate_prediction_metrics_final(results: Dict) -> pd.DataFrame:
    """Calculate comprehensive prediction metrics"""
    
    predictions_data = results['predictions']
    
    if not predictions_data:
        return pd.DataFrame()
    
    metrics_list = []
    
    # Get all model names
    all_models = set()
    for p in predictions_data:
        all_models.update(p['predictions'].keys())
    
    for model in all_models:
        actuals = []
        predictions = []
        
        for p in predictions_data:
            if model in p['predictions']:
                actuals.append(p['actual'])
                predictions.append(p['predictions'][model]['prediction'])
        
        if len(actuals) > 0:
            actuals = np.array(actuals)
            predictions = np.array(predictions)
            
            mae = np.mean(np.abs(predictions - actuals))
            mse = np.mean((predictions - actuals) ** 2)
            rmse = np.sqrt(mse)
            mape = np.mean(np.abs((predictions - actuals) / actuals)) * 100
            
            # Additional metrics
            max_error = np.max(np.abs(predictions - actuals))
            bias = np.mean(predictions - actuals)
            
            # Direction accuracy (did it predict the right trend?)
            if len(actuals) > 1:
                actual_direction = np.sign(np.diff(actuals))
                pred_direction = np.sign(np.diff(predictions))
                direction_accuracy = np.mean(actual_direction == pred_direction) * 100
            else:
                direction_accuracy = np.nan
            
            metrics_list.append({
                'Model': model,
                'MAE': round(mae, 4),
                'MSE': round(mse, 4),
                'RMSE': round(rmse, 4),
                'MAPE (%)': round(mape, 2),
                'Max Error': round(max_error, 4),
                'Bias': round(bias, 4),
                'Direction Accuracy (%)': round(direction_accuracy, 1) if not np.isnan(direction_accuracy) else 'N/A',
                'Predictions': len(predictions)
            })
    
    df_metrics = pd.DataFrame(metrics_list)
    
    # Sort by MAPE (best performance first)
    if not df_metrics.empty:
        df_metrics = df_metrics.sort_values('MAPE (%)')
    
    return df_metrics


def run_usdinr_prediction_final(
    csv_file_path: str,
    n_days: int = 4,
    context_length: int = 30,
    target_column: str = 'OPEN PRICE',
    date_column: str = 'DATE',
    save_results: bool = True
) -> Dict:
    """
    Complete pipeline for USDINR prediction with corrected CSV handling
    """
    
    print(f"Loading USDINR data from {csv_file_path}...")
    df = load_and_prepare_yfinance_data_corrected(csv_file_path)
    
    print(f"\nPredicting last {n_days} days using context of {context_length} days...")
    
    # Run predictions
    results = predict_last_n_days_with_ensemble_final(
        df=df,
        n_days=n_days,
        context_length=context_length,
        target_column=target_column,
        date_column=date_column
    )
    
    print("\nCreating visualizations...")
    
    # Create plots
    fig = create_prediction_plots_final(results, save_plots=save_results)
    
    # Calculate metrics
    metrics_df = calculate_prediction_metrics_final(results)
    print("\nPrediction Metrics (sorted by performance):")
    print(metrics_df.to_string(index=False))
    
    # Save detailed results
    if save_results:
        results_df = pd.DataFrame([
            {
                'Date': p['date'].strftime('%Y-%m-%d'),
                'Actual': p['actual'],
                **{f"{model}_pred": pred_data['prediction'] 
                   for model, pred_data in p['predictions'].items()}
            }
            for p in results['predictions']
        ])
        
        results_df.to_csv(f'USDINR_prediction_results_{n_days}days_final.csv', index=False)
        metrics_df.to_csv(f'USDINR_prediction_metrics_{n_days}days_final.csv', index=False)
        
        # Save summary report
        with open(f'USDINR_prediction_summary_{n_days}days.txt', 'w') as f:
            f.write("=== USDINR PREDICTION ANALYSIS SUMMARY ===\n\n")
            f.write(f"Analysis Period: Last {n_days} days\n")
            f.write(f"Context Length: {context_length} days\n")
            f.write(f"Data Range: {df['DATE'].min()} to {df['DATE'].max()}\n")
            f.write(f"Total Data Points: {len(df)}\n\n")
            
            f.write("MODEL PERFORMANCE RANKING:\n")
            f.write(metrics_df[['Model', 'MAPE (%)', 'MAE', 'Direction Accuracy (%)']].to_string(index=False))
            f.write("\n\n")
            
            if results['predictions']:
                best_model = metrics_df.iloc[0]['Model']
                f.write(f"BEST PERFORMING MODEL: {best_model}\n")
                f.write(f"Best Model MAPE: {metrics_df.iloc[0]['MAPE (%)']}%\n")

        print("\nResults saved to CSV files and summary report")

    return {
        'results': results,
        'figure': fig,
        'metrics': metrics_df,
        'predictions_df': results_df if save_results else None,
        'data': df
    }


# Usage example
if __name__ == "__main__":
    
    # Configuration for your specific CSV format
    config = {
        'csv_file_path': 'USDINR_daily.csv',
        'n_days': 2,
        'context_length': 30,
        'target_column': 'OPEN PRICE',
        'date_column': 'DATE',
        'save_results': True
    }
    
    # Run complete analysis
    print("Starting USDINR Prediction Analysis...")
    print("="*50)
    
    analysis_results = run_usdinr_prediction_final(**config)
    
    # Display the interactive plot
    analysis_results['figure'].show()
    
    print("\n" + "="*60)
    print("✅ ANALYSIS COMPLETE")
    print("="*60)
    print(f"✓ Loaded {len(analysis_results['data'])} days of USDINR data")
    print(f"✓ Predicted last {config['n_days']} days")
    print("✓ Interactive plot created and saved")
    print("✓ Comprehensive metrics calculated")
    print("✓ Results saved to CSV files")
    print("✓ Summary report generated")
    
    # Print best model
    if not analysis_results['metrics'].empty:
        best_model = analysis_results['metrics'].iloc[0]
        print(f"\n🏆 BEST MODEL: {best_model['Model']}")
        print(f"   MAPE: {best_model['MAPE (%)']}%")
        print(f"   MAE: {best_model['MAE']}")
        print(f"   Direction Accuracy: {best_model['Direction Accuracy (%)']}")


Starting USDINR Prediction Analysis...
Loading USDINR data from USDINR_daily.csv...
Loaded 1485 rows of data from 2020-01-01 00:00:00 to 2025-09-12 00:00:00
Available columns: ['DATE', 'CLOSE PRICE', 'HIGH PRICE', 'LOW PRICE', 'OPEN PRICE', 'Volume']
Target column dtype: float64
Sample data:
        DATE  OPEN PRICE  HIGH PRICE  LOW PRICE  CLOSE PRICE
0 2020-01-01   71.275803   71.430000  71.029999    71.275803
1 2020-01-02   71.025002   71.405502  71.025002    71.025002
2 2020-01-03   71.415001   71.995003  71.229797    71.415001
3 2020-01-06   71.720001   72.190002  71.703003    71.731003
4 2020-01-07   72.019997   72.032600  71.668999    71.787003

Predicting last 2 days using context of 30 days...
Predicting day 1/2: 2025-09-11
  Context data shape: (30,), dtype: float32
  ✓ tiny: 88.0511
  ✓ mini: 88.1154
  ✓ small: 87.9547
  ✓ base: 87.9869
  🎯 Ensemble: 88.0142 (Actual: 88.0534)
Predicting day 2/2: 2025-09-12
  Context data shape: (30,), dtype: float32
  ✓ tiny: 87.9982
  ✓ mini


✅ ANALYSIS COMPLETE
✓ Loaded 1485 days of USDINR data
✓ Predicted last 2 days
✓ Interactive plot created and saved
✓ Comprehensive metrics calculated
✓ Results saved to CSV files
✓ Summary report generated

🏆 BEST MODEL: chronos_tiny
   MAPE: 0.15%
   MAE: 0.1358
   Direction Accuracy: 0.0
